In [1]:
import numpy as np
import cv2
import glob
print("Header files loaded!")

Header files loaded!


In [2]:
path_of_images = "/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/*"
files  = glob.glob(path_of_images)
files = sorted(files)
print(files)

['/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0001.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0002.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0003.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0004.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0005.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0006.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0007.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0008.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0009.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0010.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0011.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0012.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0013.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/00

In [6]:
# get the grayscale image after converting it to lab colorspace
def get_grayscale_image(image):
    """
    Inputs:
    
    image: the input color image
    
    Outputs:
    
    gray: grayscale image
    """
    
    # convert to lab space, apply clahe to b channel and then get grayscale image
    #clahe = cv2.createCLAHE(clipLimit = 1.0, tileGridSize = (1, 1))
    #lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    #l, a, b = cv2.split(lab)
    #lab = cv2.merge((l, a, clahe.apply(b)))
    #lab = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # return grayscale image
    return gray


# update the grayscale image, that is, normalize the pixel values with template image
def update_grayscale_image(template_image, image):
    """
    Inputs:
    
    template_image: the template grayscale image whose ROI is given to us
    image: the current grayscale frame 
    
    Outputs:
    
    image: the normalized current grayscale frame
    """
    
    # get the mean of the template image and current frame and normalize
    template_mean = np.mean(template_image)
    mean = np.mean(image)
    image = (image * (template_mean / mean)).astype(float)
    
    # return the normalized current grayscale frame
    return image


# computes error between images
def compute_error_images(image_array1, image_array2):
    """
    Inputs:
    
    image_array1: the template image array
    image_array2: the current frame array
    
    Outputs:
    
    error: the error between each pixel of the template image and the current image
    """
    
    return (image_array1 - image_array2)


# computes the coordinates of the ROI of interest
def get_coordinates_array(x_range, y_range):
    """
    Inputs:
    
    x_range: array of two elements consisting of min_x and max_x of the ROI region
    y_range: array of two elements consisting of min_y and max_y of the ROI region
    
    Outputs:
    
    coordinates_array: array of all the coordinates in the ROI region of the image
    """
    
    # create coordinates array and push each possible coordinate in the ROI region
    coordinates_array = np.zeros((3, int((x_range[1] - x_range[0] + 1) * (y_range[1] - y_range[0] + 1))))
    count = 0
    for y in range(int(y_range[0]), int(y_range[1]) + 1):
        for x in range(int(x_range[0]), int(x_range[1]) + 1):
            coordinates_array[0, count] = x
            coordinates_array[1, count] = y
            coordinates_array[2, count] = 1
            count = count + 1
            
    # return the array
    return coordinates_array


# computes the new coordinates of the image wrt template image
def get_new_image_coordinates(template_image_coordinates_array, p, x_range, y_range):
    """
    Inputs:
    
    template_image_coordinates_array: the coordinates of the ROI region of template image
    p: the matrix to calculate the warped image
    x_range: the range of x values for the image
    y_range: the range of y values for the image
    
    Outputs:
    
    new_gray_image_coordinates_array: the current frame ROI coordinates
    new_rectangle_coordinates: the current frame rectangle coordinates (four: top-left, bottom-left, bottom-right, top-right)
    """
    
    # define the rectangle coordinates of the ROI of template image
    template_rectangle_coordinates = np.array([[x_range[0], x_range[0], x_range[1], x_range[1]], [y_range[0], y_range[1], y_range[1], y_range[0]], [1, 1, 1, 1]])

    # get the affine matrix to get the warped image
    affine_matrix = np.zeros((2, 3))
    count = 0
    for i in range(0, 3):
        for j in range(0, 2):
            affine_matrix[j, i] = p[count, 0]
            count = count + 1
    affine_matrix[0, 0] = affine_matrix[0, 0] + 1
    affine_matrix[1, 1] = affine_matrix[1, 1] + 1
    
    # get new rectange coordinates
    new_rectangle_coordinates = np.dot(affine_matrix, template_rectangle_coordinates)
    
    # get new coordinates
    new_gray_image_coordinates_array = np.dot(affine_matrix, template_image_coordinates_array)
    new_gray_image_coordinates_array = new_gray_image_coordinates_array.astype(int)
    
    # return the two arrays
    return (new_gray_image_coordinates_array, new_rectangle_coordinates)
    
    
# computes the pixel array
def get_pixel_array(image, image_coordinates_array):
    """
    Inputs:
    
    image: the input image
    image_coordinates_array: the ROI region in the image
    
    Outputs:
    
    pixel_array: the array consisting of pixels of the ROI region of the image
    """
    
    # get the pixel values of the ROI of the image
    pixel_array = np.zeros((1, int(image_coordinates_array.shape[1])))
    pixel_array[0, :] = image[image_coordinates_array[1, :], image_coordinates_array[0, :]]
    
    # return the pixel array
    return pixel_array
    
    
# compute the error in the p matrix
def get_delta_p(error, steep_descent):
    """
    Inputs:
    
    error: the error between images
    steep_descent: the steep descent value
    
    Outputs:
    delta_p: the change in the p matrix
    """
    
    # compute the sd_param and hessian matrix
    sd_param_matrix = np.dot(steep_descent.T, error.T)
    hessian_matrix = np.dot(steep_descent.T, steep_descent)
    hessian_matrix_inv = np.linalg.pinv(hessian_matrix)
    
    # use the above two matrices to get the error in p matrix and return
    delta_p = np.dot(hessian_matrix_inv, sd_param_matrix)
    return delta_p    
    
    
# compute the steep descent using two images and the coordinates of the ROI of two images
def get_steep_descent(sobelx, sobely, template_image_coordinates_array, new_gray_image_coordinates_array):
    """
    Inputs:
    
    sobelx: the derivative along x-direction
    sobely: the derivative along y-direction
    template_gray_image_coordinates_array: the ROI coordinates of the template image
    new_gray_image_coordinates_array: the ROI coordinates of the current frame
    
    Outputs:
    
    image: 6 images formed using above information
    """
    
    # get the pixel array for sobelx
    sobelx_pixel_array = get_pixel_array(sobelx, new_gray_image_coordinates_array)
    
    # get the pixel array for sobely
    sobely_pixel_array = get_pixel_array(sobely, new_gray_image_coordinates_array)
    
    # get four images
    image1 = sobelx_pixel_array * template_image_coordinates_array[0, :]
    image2 = sobely_pixel_array * template_image_coordinates_array[0, :]
    image3 = sobelx_pixel_array * template_image_coordinates_array[1, :]
    image4 = sobely_pixel_array * template_image_coordinates_array[1, :]
    
    # return the six images
    return np.vstack((image1, image2, image3, image4, sobelx_pixel_array, sobely_pixel_array)).T
    
    
# lucas kanade algorithm
def lucas_kanade_algorithm(template_gray_image, current_gray_image, x_range, y_range, thresh, p):
    """
    Inputs:
    
    template_gray_image: the grayscale template image
    current_gray_image: the grayscale current frame
    x_range: the array consisting of min_x and max_x values
    y_range: the array consisting of min_y and max_y values
    thresh: the threshold after which we need to break the loop
    p: the matrix used for calculating the warped image
    
    Outputs:
    
    new_rectangle_coordinates: the new ROI for the current frame
    """
    
    # get the coordinates of the ROI for template image
    template_image_coordinates_array = get_coordinates_array(x_range, y_range)
    
    # define p matrix, used for calculating the warped image for template image
    p_template = np.array([[0, 0, 0, 0, 0, 0]]).T
    
    # get the coordinates of the ROI in the new frame
    (new_template_image_coordinates_array, new_rectangle_coordinates) = get_new_image_coordinates(template_image_coordinates_array, p_template, x_range, y_range)
    
    # get the pixel array of the template image
    template_pixel_array = get_pixel_array(template_gray_image, new_template_image_coordinates_array)
    
    # compute derivatives around x and y directions for the current frame
    sobelx = cv2.Sobel(current_gray_image, cv2.CV_64F, 1, 0, ksize = 3)
    sobely = cv2.Sobel(current_gray_image, cv2.CV_64F, 0, 1, ksize = 3)
    
    # run algorithm
    count = 0
    while(True):
        
        # get the coordinates of the ROI in the new frame
        (new_gray_image_coordinates_array, new_rectangle_coordinates) = get_new_image_coordinates(template_image_coordinates_array, p, x_range, y_range)
        
        # if new coordinates not in range, then break
        if(count > 10 or new_gray_image_coordinates_array[0, 0] < 0 or new_gray_image_coordinates_array[1, 0] < 0 or new_gray_image_coordinates_array[0, new_gray_image_coordinates_array.shape[1] - 1] > current_gray_image.shape[1] or new_gray_image_coordinates_array[1, new_gray_image_coordinates_array.shape[1] - 1] > current_gray_image.shape[0]):
            break
            
        # get the pixel array of the gray image
        new_pixel_array = get_pixel_array(current_gray_image, new_gray_image_coordinates_array)
        
        # compute the error
        error = compute_error_images(template_pixel_array, new_pixel_array)
        
        # compute steep descent
        steep_descent = get_steep_descent(sobelx, sobely, template_image_coordinates_array, new_gray_image_coordinates_array)
        
        # get the delta_p
        delta_p = get_delta_p(error, steep_descent)
        
        # get p norm and update p matrix
        p_norm = np.linalg.norm(delta_p)
        p = np.reshape(p, (6, 1))
        p = p + delta_p
        count = count + 1
        
        # if p_norm within thresh break
        if(p_norm < thresh):
            break
            
    # return the ROI needed for this frame
    return (new_rectangle_coordinates, p)

In [7]:
threshold = 0.01
x_range = np.array([269, 303])
y_range = np.array([75, 139])
p = np.array([[0, 0, 0, 0, 0, 0]]).T
template_image = cv2.imread(files[0])
gray_template_image = get_grayscale_image(template_image)

In [8]:
count = 0

# read files
for file in files:
    # read image
    image = cv2.imread(file)
    image_copy = image.copy()
    
    # get grayscale image
    gray = get_grayscale_image(image_copy)
    gray = update_grayscale_image(gray_template_image, gray)
    
    (new_rectangle_coordinates, p) = lucas_kanade_algorithm(gray_template_image, gray, x_range, y_range, threshold, p)
    print(new_rectangle_coordinates)
    count = count + 1
    
    image = cv2.rectangle(image, (int(new_rectangle_coordinates[0, 0]), int(new_rectangle_coordinates[1, 0])), (int(new_rectangle_coordinates[0, 2]), int(new_rectangle_coordinates[1, 2])), (0, 0, 255), 2)
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

[[269. 269. 303. 303.]
 [ 75. 139. 139.  75.]]
[[269.46857319 268.10601832 301.85484459 303.21739946]
 [ 74.30374072 139.14844309 138.97298343  74.12828106]]
[[269.96072602 267.51484848 300.7644755  303.21035305]
 [ 72.88314257 139.45418262 139.15787097  72.58683092]]
[[268.67389958 268.23114097 301.23452058 301.67727919]
 [ 72.46386503 140.22487656 139.34324183  71.5822303 ]]
[[266.81617666 269.27936893 303.07227932 300.60908705]
 [ 71.89295978 140.57925695 139.29162965  70.60533249]]
[[265.2140128  269.13006682 303.75348558 299.83743156]
 [ 71.94576763 140.05540535 139.00181178  70.89217407]]
[[265.1694548  270.17726185 304.04903228 299.04122523]
 [ 72.24504236 140.00080676 138.92648682  71.17072242]]
[[265.08890516 269.52314976 303.52390367 299.08965908]
 [ 72.40452201 139.5167805  138.68343919  71.5711807 ]]
[[264.28086492 268.46998813 303.68132552 299.4922023 ]
 [ 72.14290656 138.65335026 138.71547924  72.20503554]]
[[263.45617973 266.15878463 303.39542167 300.69281677]
 [ 73.0292

[[257.5702083  271.65026593 316.95842243 302.8783648 ]
 [ 57.02687552 147.85069134 160.52657148  69.70275567]]
[[259.10140196 271.54115642 314.26318914 301.82343468]
 [ 56.56012793 149.1654785  161.98207803  69.37672746]]
[[258.7894199  273.68298501 314.03662566 299.14306055]
 [ 56.51249676 149.50009289 162.69540222  69.70780609]]
[[259.86076874 274.18287219 313.02143394 298.69933049]
 [ 57.34027056 149.58781831 162.56242563  70.31487788]]
[[260.78290504 275.20869999 312.09934413 297.67354918]
 [ 58.82486581 150.68425237 162.34971127  70.49032471]]
[[262.10676452 276.30855479 311.45911813 297.25732786]
 [ 59.99648734 151.14190279 161.71303787  70.56762242]]
[[261.99136568 276.65777236 311.392774   296.72636732]
 [ 59.71315393 151.91261834 162.65051335  70.45104893]]
[[262.15906742 276.14413744 310.77368534 296.78861533]
 [ 59.10377066 152.23682455 163.42288596  70.28983207]]
[[261.39883058 276.09181998 311.3693277  296.6763383 ]
 [ 58.69338971 152.86639178 164.05450148  69.88149941]]
[

[[245.50261605 299.38137076 330.66198194 276.78322723]
 [ 60.69302608 159.7300531  162.29202309  63.25499607]]
[[245.61016898 299.52580531 329.48583481 275.57019849]
 [ 60.31506273 159.18638945 163.02217153  64.15084481]]
[[242.63884861 301.24087614 332.4950719  273.89304437]
 [ 60.24064607 158.07156604 163.27273958  65.44181962]]
[[240.13686161 301.43970377 334.84770942 273.54486726]
 [ 60.81137376 157.47815939 162.74336303  66.0765774 ]]
[[239.4067636  300.65277713 335.37399181 274.12797828]
 [ 60.08827119 157.68135522 163.31891846  65.72583443]]
[[244.1145425  301.19702495 332.18882024 275.10633779]
 [ 58.45269463 158.169584   164.19953014  64.48264077]]
[[247.21425466 302.05935454 330.45439668 275.6092968 ]
 [ 57.89311384 157.76307204 164.05293667  64.18297847]]
[[248.94881796 302.13211824 329.4028787  276.21957842]
 [ 56.41922031 158.10095983 164.87736631  63.19562679]]
[[248.51535694 303.22154027 329.70328126 274.99709793]
 [ 55.59453463 158.2034718  165.57631521  62.96737803]]
[

[[293.16077082 306.17145484 306.59165327 293.58096926]
 [ 54.56897367 153.0461807  166.14630252  67.66909549]]
[[293.57833741 307.02426302 306.28827871 292.8423531 ]
 [ 54.07743631 152.98464322 166.43565367  67.52844676]]
[[293.66827457 307.04702801 306.65938539 293.28063194]
 [ 54.11880339 153.07684095 166.35141271  67.39337515]]
[[291.93016788 309.52431022 310.03323324 292.43909091]
 [ 53.82917396 153.23716876 166.5873474   67.1793526 ]]
[[291.99918779 309.18541472 310.01508776 292.82886083]
 [ 53.65506552 152.68850315 166.69457014  67.66113251]]
[[292.5821498  308.83057346 309.89709199 293.64866833]
 [ 53.47841167 152.33532779 166.66978493  67.81286881]]
[[293.99557712 306.99072043 307.81295741 294.8178141 ]
 [ 53.7008177  151.88741014 166.75359974  68.56700729]]
[[290.09648964 311.16313312 313.36511093 292.29846745]
 [ 53.42686722 152.085238   167.17043701  68.51206623]]
[[286.77301957 311.44116067 315.32368484 290.65554374]
 [ 53.49419665 152.13063456 167.37447328  68.73803537]]
[

In [ ]:
np.ceil(2.2)